In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



This notebook shows a "most popular local hotel" benchmark implemented with pandas.

### Read the train data

Read in the train data using only the necessary columns. 
Specifying dtypes helps reduce memory requirements. 

The file is read in chunks of 1 million rows each. In each chunk we count the number of rows and number of bookings for every destination-hotel cluster combination.

In [2]:
train = pd.read_csv('data/train.csv',
                    dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32},
                    usecols=['srch_destination_id','is_booking','hotel_cluster'],
                    chunksize=1000000)
aggs = []
print('-'*38)
for chunk in train:
    agg = chunk.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)
    print('.',end='')
print('')
aggs = pd.concat(aggs, axis=0)
aggs.head()

--------------------------------------
......................................


,srch_destination_id,hotel_cluster,sum,count
0,1,20,0,2
1,1,30,0,1
2,1,60,0,2
3,4,22,1,2
4,4,25,1,2


In [20]:
aggs.shape

(3632017, 4)

Next we aggregate again to compute the total number of bookings over all chunks. 

Compute the number of clicks by subtracting the number of bookings from total row counts.

Compute the 'relevance' of a hotel cluster with a weighted sum of bookings and clicks.

In [19]:
CLICK_WEIGHT = 0.05
agg = aggs.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,0,3,0,2,0.10
1,1,20,4,22,5.10
2,1,30,2,20,3.00
3,1,57,0,1,0.05
4,1,60,0,17,0.85


### Find most popular hotel clusters by destination

Define a function to get most popular hotels for a destination group.

Previous version used nlargest() Series method to get indices of largest elements. 
But as @benjamin points out [in his fork](https://www.kaggle.com/benjaminabel/expedia-hotel-recommendations/pandas-version-of-most-popular-hotels/comments) the method is rather slow. 
I have updated this notebook with a version that runs faster.

In [21]:
most_pop.shape

(59455, 1)

In [22]:
def most_popular(group, n_max=10):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

Get most popular hotel clusters for all destinations.

In [23]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
srch_destination_id,
0,3
1,20 30 60 57
2,20 30 53 46 41 38 5
3,53 60
4,82 25 32 58 78 30 67 22 81 38


In [24]:
most_pop.tail()

,hotel_cluster
srch_destination_id,
65098,94 50
65102,51 7
65103,42 50 20 7
65104,37 5
65107,76


### Predict for test data
Read in the test data and merge most popular hotel clusters.

In [7]:
test = pd.read_csv('data/test.csv',
                    dtype={'srch_destination_id':np.int32},
                    usecols=['srch_destination_id'],)

In [8]:
test = test.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)
test.head()

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5
2,11353,0 31 77 91 96
3,8250,1 45 79 24 54
4,11812,91 42 2 48 59


Check hotel_cluster column in test for null values.

In [9]:
test.hotel_cluster.isnull().sum()

14036

Looks like there's about 14k new destinations in test. Let's fill nas with hotel clusters that are most popular overall.

In [10]:
most_pop_all = agg.groupby('hotel_cluster')['relevance'].sum().nlargest(5).index
most_pop_all = np.array_str(most_pop_all)[1:-1]
most_pop_all

'91 48 42 59 28'

In [11]:
test.hotel_cluster.fillna(most_pop_all,inplace=True)

Save the submission.

In [ ]:
test.hotel_cluster.to_csv('predicted_with_pandas.csv',header=True, index_label='id')

In [ ]:
test

test

In [ ]:
test.hotel_cluster.to_csv('predicted_with_pandas.csv',header=True, index_label='id')

In [13]:
test

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5
2,11353,0 31 77 91 96
3,8250,1 45 79 24 54
4,11812,91 42 2 48 59
...,...,...
2528238,8857,26 73 0 84 34
2528239,19308,57
2528240,8250,1 45 79 24 54
2528241,4627,50 47 43 15 32
